In [ ]:
import librosa 
from scipy.signal import lfilter
import numpy as np
from librosa import mel_frequencies
import warnings
import matplotlib.pyplot as plt

# Plotting waveforms

In [ ]:
# plot some example audio 
nrows = 20
ncols = 12
zoom = 2
fig, axs = plt.subplots(ncols=ncols, nrows = nrows,figsize = (ncols*zoom, nrows+zoom/1.5))
for i, syll in tqdm(enumerate(syllable_df['audio'].values), total = nrows*ncols):
    ax = axs.flatten()[i]
    ax.plot(syll)
    if i == nrows*ncols -1:
        break

# Define spectrogram builder

In [ ]:
def make_spec(
    syll_wav,
    fs,
    hparams,
    mel_matrix=None,
    use_tensorflow=False,
    use_mel=True,
    return_tensor=False,
    norm_uint8=False,
):
    """
    """
    if use_tensorflow:
        import tensorflow as tf
        from avgn.signalprocessing.spectrogramming_tf import spectrogram_tensorflow
    # convert to float
    if type(syll_wav[0]) == int:
        syll_wav = int16_to_float32(syll_wav)

    # create spec
    if use_tensorflow:
        spec = spectrogram_tensorflow(syll_wav, fs, hparams)
        if use_mel:
            spec = tf.transpose(tf.tensordot(spec, mel_matrix, 1))
            if not return_tensor:
                spec = spec.numpy()
    else:
        spec = spectrogram(syll_wav, fs, hparams)
        if use_mel:
            spec = np.dot(spec.T, mel_matrix).T
    if norm_uint8:
        spec = (norm(spec) * 255).astype("uint8")

    return spec

Helper functions

In [ ]:
def spectrogram(y, fs, hparams):
    return _normalize(spectrogram_nn(y, fs, hparams), hparams)

def spectrogram_nn(y, fs, hparams):
    D = _stft(preemphasis(y, hparams), fs, hparams)
    S = _amp_to_db(np.abs(D)) - hparams.ref_level_db
    return S

def preemphasis(x, hparams):
    return signal.lfilter([1, -hparams.preemphasis], [1], x)

def _amp_to_db(x):
    return 20 * np.log10(np.maximum(1e-5, x))

def _normalize(S, hparams):
    return np.clip((S - hparams.min_level_db) / -hparams.min_level_db, 0, 1)

def _stft(y, fs, hparams):
    return librosa.stft(
        y=y,
        n_fft=hparams.n_fft,
        hop_length=int(hparams.hop_length_ms / 1000 * fs),
        win_length=int(hparams.win_length_ms / 1000 * fs),
    )

Loop with parallel

In [ ]:
with Parallel(n_jobs=n_jobs, verbose=verbosity) as parallel:
    # create spectrograms
    syllables_spec = parallel(
        delayed(make_spec)(
            syllable,
            rate,
            hparams=dataset.hparams,
            mel_matrix=dataset.mel_matrix,
            use_mel=True,
            use_tensorflow=False,
        )
        for syllable, rate in tqdm(
            zip(syllable_df.audio.values, syllable_df.rate.values),
            total=len(syllable_df),
            desc="getting syllable spectrograms",
            leave=False,
        )
    )

Test print a spectrogram

In [ ]:
# plot an example syllable
import matplotlib.pyplot as plt
plt.matshow(syllables_spec[100])

# Defining spectrogram printer

In [ ]:
def draw_spec_set(
    spectrograms,
    maxrows=3,
    colsize=10,
    cmap=plt.cm.afmhot,
    zoom=2,
    ax=None,
    axis_off=True,
):
    """
    """
    # get column and row sizes
    rowsize = np.shape(spectrograms[0])[0]
    colsize = colsize * np.shape(spectrograms[0])[1]

    # create the vanvas
    canvas = np.zeros((rowsize * maxrows, colsize))

    # fill the canvas
    column_pos = 0
    row = 0
    for speci, spec in enumerate(spectrograms):
        spec_shape = np.shape(spec)
        if column_pos + spec_shape[1] > colsize:
            if row == maxrows - 1:
                break
            row += 1
            column_pos = 0
        canvas[
            rowsize * (maxrows - 1 - row) : rowsize * ((maxrows - 1 - row) + 1),
            column_pos : column_pos + spec_shape[1],
        ] = spec
        column_pos += spec_shape[1]
    if row < maxrows - 1:
        canvas = canvas[(maxrows - 1 - row) * rowsize :, :]
    # print(speci)
    figsize = (zoom * (colsize / rowsize), zoom * (row + 1))
    # print(figsize, np.shape(canvas), colsize / rowsize, rowsize, colsize)
    if ax is None:
        fig, ax = plt.subplots(figsize=figsize)

    ax.matshow(
        canvas, cmap=cmap, origin="lower", aspect="auto", interpolation="nearest"
    )
    if axis_off:
        ax.axis("off")
    if ax is None:
        plt.show()